In [16]:
import requests
import zipfile
import re
import math
from collections import Counter
import random

In [10]:
r = requests.get("https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/smsspamcollection.zip")
with open("sms.zip", "wb") as f:
  f.write(r.content)
zipfile.ZipFile("sms.zip").extractall()

In [18]:
#데이터 전처리
data = []
with open("SMSSpamCollection.txt", "r") as f:
  for line in f:
    data.append(line.strip().split("\t"))

for d in data:
  #d[1] -> 내용
  d[1] = set(re.findall("[a-z0-1_]+", d[1].lower()))
  #print(d[1])

In [19]:
#train, test 나누기
random.shuffle(data)

train_size = int(len(data) * 0.8)
test_size = len(data) - train_size 

train = data[:train_size]
test = data[train_size:]

print(len(train), len(test))

4459 1115


In [31]:
#navie bayes
#사전확률 구하기

n_total = len(train)
n_spam = sum(1 for cls, bow in train if cls == 'spam') 
n_ham = sum(1 for cls, bow in train if cls == 'ham')

alpha = 0.0001

prior_spam = (n_spam +alpha) /(n_total + 2*alpha)
prior_ham = (n_ham +alpha) / (n_total + 2*alpha)

In [25]:
#각 단어의 가능도 구하기

#w를 포함하는 스팸 수, 햄 수

# spam_words = {}
# ham_words = {}

# for cls, bow in train:
#   for word in bow:
#     if cls == 'spam':
#       if word not in spam_words:
#         spam_words[word] = 0
#       spam_words[word] += 1
#     if cls == 'ham':
#       if word not in ham_words:
#         ham_words[word] = 0
#       ham_words[word] += 1

#더 간단하게 

ham_words = Counter(word for cls, bow in train
                      for word in bow
                      if cls == 'ham')
spam_words = Counter(word for cls, bow in train
                      for word in bow
                     if cls=='spam')

In [35]:
def predict(bow, ham_words, spam_words, n_spams, n_hams, prior_spam, prior_ham, alpha):
  spam_score = math.log(prior_spam)

  for word in bow:
    spam_score += math.log((spam_words[word] + alpha) / (n_spams + alpha))

  ham_score = math.log(prior_ham)

  for word in bow:
    ham_score += math.log((ham_words[word] + alpha) / (n_hams +alpha))
  
  if spam_score < ham_score:
    return "ham"
  else:
    return "spam"

In [36]:
tp,tn,fp,fn = 0,0,0,0

for ans, bow in test:
  pred = predict(bow, ham_words, spam_words, n_spam, n_ham, prior_spam, prior_ham, alpha)

  if pred == 'spam':
    if ans == 'spam':
      tp += 1
    else:
      fp += 1
  else:
    if ans == 'ham':
      tn += 1
    else:
      fn += 1
print("tp", tp)
print("tn", tn)
print("fp", fp)
print("fn", fn)
    

tp 136
tn 942
fp 32
fn 5


In [37]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2*precision*recall/(precision+recall)
print("accuracy", accuracy)
print("precision", precision)
print("recall", recall)
print("f1", f1)

accuracy 0.9668161434977578
precision 0.8095238095238095
recall 0.9645390070921985
f1 0.8802588996763755
